In [60]:
import pandas as pd
import os

# Определение путей к файлам
ratings_file = "/content/u.data"
movies_file = "/content/u.item"
users_file = "/content/u.user"

In [61]:
# Определение названий колонок
ratings_columns = ['user_id', 'item_id', 'rating', 'timestamp']
movies_columns = ['item_id', 'title']
users_columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']

# Загрузка данных
ratings = pd.read_csv(ratings_file, sep='\t', names=ratings_columns, encoding='latin-1')
movies = pd.read_csv(movies_file, sep='|', names=movies_columns, usecols=[0, 1], encoding='latin-1')
users = pd.read_csv(users_file, sep='|', names=users_columns, encoding='latin-1')

# Объединение данных о рейтингах, фильмах и пользователях
data = pd.merge(ratings, movies, on='item_id')
data = pd.merge(data, users, on='user_id')

# Вывод структуры данных
print("Структура данных:")
print(data.head())

# Проверка и удаление пропущенных значений
print(f"\nКоличество пропущенных значений:\n{data.isnull().sum()}")
data.dropna(inplace=True)

# Фильтрация неактивных пользователей и фильмов
# Пользователи с менее чем 5 оценками удаляются
active_users = data.groupby('user_id').size()
data = data[data['user_id'].isin(active_users[active_users >= 5].index)]

# Фильмы с менее чем 3 оценками удаляются
popular_movies = data.groupby('item_id').size()
data = data[data['item_id'].isin(popular_movies[popular_movies >= 3].index)]

# Вывод итогового размера данных
print(f"\nРазмер очищенного набора данных: {data.shape}")

# Сохранение обработанных данных
processed_file = "/content/processed_movielens.csv"
data.to_csv(processed_file, index=False)


Структура данных:
   user_id  item_id  rating  timestamp                       title  age  \
0      196      242       3  881250949                Kolya (1996)   49   
1      186      302       3  891717742    L.A. Confidential (1997)   39   
2       22      377       1  878887116         Heavyweights (1994)   25   
3      244       51       2  880606923  Legends of the Fall (1994)   28   
4      166      346       1  886397596         Jackie Brown (1997)   47   

  gender  occupation zip_code  
0      M      writer    55105  
1      F   executive    00000  
2      M      writer    40206  
3      M  technician    80525  
4      M    educator    55113  

Количество пропущенных значений:
user_id       0
item_id       0
rating        0
timestamp     0
title         0
age           0
gender        0
occupation    0
zip_code      0
dtype: int64

Размер очищенного набора данных: (99723, 9)


In [47]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Загрузка обработанного датасета
data_path = "/content/processed_movielens.csv"
data = pd.read_csv(data_path)

2.1. Построить матрицу взаимодействий (User-Item), где строки — это пользователи, а столбцы — это товары (например, фильмы), значения — оценки или количество взаимодействий.

2.2. Реализовать алгоритм коллаборативной фильтрации на основе сходства пользователей или товаров:

**User-based:** Находить похожих пользователей на основе их оценок.

**Item-based:** Находить похожие товары на основе оценок пользователей.


In [48]:
# Создание матрицы взаимодействий (пользователи x фильмы)
interaction_matrix = data.pivot_table(index='user_id', columns='title', values='rating')

# Заполнение пропусков средним значением по строкам
interaction_matrix = interaction_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)

# --- User-based коллаборативная фильтрация ---
# Вычисление сходства пользователей на основе косинусного расстояния
user_similarity = cosine_similarity(interaction_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=interaction_matrix.index, columns=interaction_matrix.index)

# --- Item-based коллаборативная фильтрация ---
# Вычисление сходства товаров на основе оценок пользователей
item_similarity = cosine_similarity(interaction_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, index=interaction_matrix.columns, columns=interaction_matrix.columns)


In [49]:
# Функция для поиска похожих пользователей
def get_similar_users(user_id, top_n=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).iloc[1:top_n+1]
    return similar_users

# Функция для поиска похожих фильмов
def get_similar_items(movie_title, top_n=5):
    similar_items = item_similarity_df[movie_title].sort_values(ascending=False).iloc[1:top_n+1]
    return similar_items

# Пример использования:
user_id = 15
movie_title = "Star Wars (1977)"

print(f"Похожие пользователи для {user_id}:\n", get_similar_users(user_id),"\n")
print(f"Похожие фильмы для {movie_title}:\n", get_similar_items(movie_title),"\n")

Похожие пользователи для 15:
 user_id
794    0.992895
482    0.992612
700    0.992502
596    0.992500
791    0.992465
Name: 15, dtype: float64 

Похожие фильмы для Star Wars (1977):
 title
Return of the Jedi (1983)          0.987279
Empire Strikes Back, The (1980)    0.986843
Raiders of the Lost Ark (1981)     0.983069
Man of the Year (1995)             0.982289
Fresh (1994)                       0.982272
Name: Star Wars (1977), dtype: float64 



3.1. Для заданного пользователя предсказать оценки для товаров, которые он ещё не оценил, на основе похожих пользователей или товаров.

3.2. Выдать пользователю список рекомендованных товаров.


In [50]:
# Функция для предсказания рейтингов
def predict_rating(user_id, movie_title):
    if movie_title not in interaction_matrix.columns:
        return None  # Фильм отсутствует в базе
    similar_users = get_similar_users(user_id, top_n=5)
    weighted_ratings = sum(interaction_matrix.loc[sim_user, movie_title] * similarity
                           for sim_user, similarity in similar_users.items() if not np.isnan(interaction_matrix.loc[sim_user, movie_title]))
    total_similarity = sum(similar_users)
    return weighted_ratings / total_similarity if total_similarity > 0 else np.nan

# Функция для выдачи рекомендаций пользователю
def recommend_movies(user_id, top_n=5):
    if user_id not in interaction_matrix.index:
        print(f"Пользователь {user_id} отсутствует в базе.")
        return []

    # Найти фильмы, которые пользователь еще не оценил
    user_ratings = interaction_matrix.loc[user_id]
    unseen_movies = user_ratings[user_ratings.isna()].index.tolist()

    if not unseen_movies:
        print(f"Пользователь {user_id} уже оценил все фильмы.")
        return []

    # Предсказать оценки для неоцененных фильмов
    predicted_ratings = {movie: predict_rating(user_id, movie) for movie in unseen_movies}

    # Удалить фильмы с NaN-значениями предсказанных оценок
    predicted_ratings = {movie: rating for movie, rating in predicted_ratings.items() if not np.isnan(rating)}

    if not predicted_ratings:
        print(f"Не удалось предсказать рейтинги для пользователя {user_id}.")
        return []

    # Отсортировать фильмы по предсказанному рейтингу
    recommended_movies = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)[:top_n]

    return [movie for movie, rating in recommended_movies]


# Пример использования:
user_id = 187
movie_title = "Star Wars (1977)"

print(f"Предсказанный рейтинг для фильма {movie_title}: {predict_rating(user_id, movie_title)}","\n")
print(f"Рекомендации для пользователя {user_id}: {recommend_movies(user_id)}","\n")

Предсказанный рейтинг для фильма Star Wars (1977): 4.368004437397854 

Пользователь 187 уже оценил все фильмы.
Рекомендации для пользователя 187: [] 



4.1. Разделить данные на обучающую и тестовую выборки.

4.2. Использовать метрики качества для оценки модели, такие как RMSE (Root Mean Squared Error) или MAE (Mean Absolute Error).

4.3. Проанализировать результаты и предложить улучшения.
Например, варьировать параметры и наблюдать, как изменяются метрики (RMSE, MAE).

5.	Проанализировать недостатки User-based подхода, такие как холодный старт и проблемы со слишком редкими данными.

6.	Перейти к Item-based коллаборативной фильтрации, сравнить результаты.


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [52]:
# Разделение данных (80% - train, 20% - test)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Создание матрицы пользователь-фильм
interaction_matrix = train_data.pivot_table(index='user_id', columns='title', values='rating')

# Вычисление средних оценок фильмов
movie_means = interaction_matrix.mean(axis=0)

3. Функция предсказания оценок

In [53]:
def predict_rating_fast(movie_title):
    return movie_means.get(movie_title, np.nan)  # Если фильма нет в данных, вернуть NaN


4. Вычисление RMSE и MAE

In [55]:
test_data['predicted_rating'] = test_data.apply(lambda row: predict_rating_fast(row['title']), axis=1)

# Удаление NaN перед расчетом метрик
test_data = test_data.dropna(subset=['predicted_rating'])

# Вычисление RMSE и MAE
rmse = np.sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
mae = mean_absolute_error(test_data['rating'], test_data['predicted_rating'])

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

RMSE: 1.0169
MAE: 0.8117


5. Анализ недостатков User-based подхода
Холодный старт: если у нового пользователя нет оценок, мы не можем сделать предсказание.
Редкость данных: если фильм имеет мало оценок, сложно вычислить среднее значение.
Выбор похожих пользователей: косинусное сходство работает плохо, если мало данных.


In [56]:
# Анализ разреженности данных
sparsity = 1 - (train_data.shape[0] / (train_data['user_id'].nunique() * train_data['title'].nunique()))
print(f"Разреженность матрицы взаимодействий: {sparsity:.2%}")

# Анализ "холодного старта" (количество пользователей и фильмов с малым числом оценок)
user_activity = train_data.groupby('user_id').size()
movie_popularity = train_data.groupby('title').size()

cold_start_users = (user_activity < 5).sum()  # Пользователи с менее 5 оценками
cold_start_movies = (movie_popularity < 5).sum()  # Фильмы с менее 5 оценками

print(f"Пользователей с недостаточным числом оценок (<5): {cold_start_users}")
print(f"Фильмов с недостаточным числом оценок (<5): {cold_start_movies}")


Разреженность матрицы взаимодействий: 94.21%
Пользователей с недостаточным числом оценок (<5): 0
Фильмов с недостаточным числом оценок (<5): 181


6. Как улучшить:
Использовать Item-based подход: рекомендации на основе схожести фильмов.
Учитывать среднюю оценку пользователя при предсказаниях.
Добавить матрицу сходства фильмов.


In [59]:
from sklearn.metrics.pairwise import cosine_similarity

# Создание матрицы "фильм x пользователь"
item_matrix = train_data.pivot_table(index='title', columns='user_id', values='rating')

# Заполнение NaN средним по фильму
item_matrix = item_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)

# Вычисление сходства между фильмами (Item-based)
item_similarity = cosine_similarity(item_matrix)
item_similarity_df = pd.DataFrame(item_similarity, index=item_matrix.index, columns=item_matrix.index)

# Функция рекомендации похожих фильмов
def get_similar_items(movie_title, top_n=5):
    if movie_title not in item_similarity_df.index:
        return []
    similar_items = item_similarity_df[movie_title].sort_values(ascending=False).iloc[1:top_n+1]
    return list(similar_items.index)

# Пример: ищем похожие фильмы для "Star Wars (1977)"
similar_movies = get_similar_items("Star Wars (1977)")
print(f"Фильмы, похожие на 'Star Wars (1977)': {similar_movies}")


Фильмы, похожие на 'Star Wars (1977)': ['Empire Strikes Back, The (1980)', 'To Be or Not to Be (1942)', 'Man of No Importance, A (1994)', 'Pather Panchali (1955)', 'Moonlight and Valentino (1995)'],

